In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow, subplots
from numpy import array, zeros, ones
from colorsys import hsv_to_rgb
from pybricks.parameters import Color
from random import randint

In [ ]:
Color.BLACK = Color(0, 0, 20, 'BLACK')

color_map = (Color.RED, Color.BLUE, Color.YELLOW, Color.GREEN, Color.BLACK, Color.WHITE, None)

In [ ]:
def pybricks_hsv_to_rgb(h, s, v):
    return [int(c*255) for c in hsv_to_rgb(h/360, s/100, v/100)]

INT32_MAX = 2147483547


def pybricks_hsv_cost(x, c):
    if x.s < 40 or x.v <= 5:
        if c.s == 100:
            return INT32_MAX
        return x.v - c.v if x.v > c.v else c.v - x.v

    hue_error = c.h - x.h if c.h > x.h else x.h - c.h;
    return hue_error if hue_error <= 180 else 360 - hue_error

def pybricks_discretize_color(measured):

    match = None
    cost_now = INT32_MAX
    cost_min = INT32_MAX

    for compare in color_map:
        
        if compare is None:
            compare = Color(0, 0, 0, 'None')

        cost_now = pybricks_hsv_cost(measured, compare)

        if cost_now < cost_min:
            cost_min = cost_now
            match = compare

    if match is not None and match.name == 'None':
        match = None
            
    return match



SATURATION_INCREMENT = 50

saturation_range = range(100, -SATURATION_INCREMENT, -SATURATION_INCREMENT)
hue_range = range(360)
value_range = range(0, 101)

figure, axes = subplots(nrows=len(saturation_range), ncols=2, figsize=(12, 6*len(saturation_range)))

image_measured = zeros((101, 360, 3), dtype=int)
image_discrete = zeros((101, 360, 4), dtype=int)
image_alphadot = ones((101, 360, 3), dtype=int)*255

for i in hue_range:
    if (i//10) % 2:
        for j in value_range:
            image_alphadot[j, i] = (180, 180, 180)


for saturation_index, saturation in enumerate(saturation_range):
    for value in value_range:
        for hue in hue_range:           
            image_measured[value][hue][:] = pybricks_hsv_to_rgb(hue, saturation, value)
            
            discrete = pybricks_discretize_color(Color(hue, saturation, value))
            if discrete is None:
                image_discrete[value][hue][:] = 0, 0, 0, 0
            else:
                r, g, b = pybricks_hsv_to_rgb(discrete.h, discrete.s, discrete.v)
                image_discrete[value][hue][:] = r, g, b, 255
    axes[saturation_index][0].imshow(image_measured, aspect=3.6, origin='lower')
    axes[saturation_index][1].imshow(image_alphadot, aspect=3.6, origin='lower')
    axes[saturation_index][1].imshow(image_discrete, aspect=3.6, origin='lower')